## Tarea CVA

In [1]:
import numpy as np
from datetime import timedelta, datetime
import calendar
from scipy.optimize import minimize
import matplotlib.pyplot as plt

Replica de la funcion Days360 que utiliza excel (Referencia de https://www.reddit.com/r/learnpython/comments/3942zm/python_implementation_of_excels_days360_function/)

In [52]:
def days360(start_date, end_date, method_eu=False):
    
    start_day = start_date.day
    start_month = start_date.month
    start_year = start_date.year
    end_day = end_date.day
    end_month = end_date.month
    end_year = end_date.year

    if (
        start_day == 31 or
        (
            method_eu is False and
            start_month == 2 and (
                start_day == 29 or (
                    start_day == 28 and
                    calendar.isleap(start_year) is False
                )
            )
        )
    ):
        start_day = 30

    if end_day == 31:
        if method_eu is False and start_day != 30:
            end_day = 1

            if end_month == 12:
                end_year += 1
                end_month = 1
            else:
                end_month += 1
        else:
            end_day = 30

    return (
        end_day + end_month * 30 + end_year * 360 -
        start_day - start_month * 30 - start_year * 360
    )

In [331]:
2+2

4

Definimos la funcion de PrecioSucio, que calcula el precio Sucio de un bono dada la fecha de valuacion, fecha de vencimiento, el nocional, ytm, tasa cupon, periodo (semestral por default), h y R (el riesgo de incumplimiento, el default es no tener riesgo)

In [94]:
def PrecioSucio(fecha_valuacion, fecha_vencimiento, nocional, ytm, tasa_cupon, periodo_cupon = 2, h=0,R=1):
    
    cupon=tasa_cupon*nocional/periodo_cupon
    ytm_periodo=ytm/periodo_cupon
    
    dias_val_venc=days360(fecha_valuacion,fecha_vencimiento)
    periodo_frac=(dias_val_venc%(360/periodo_cupon))/(360/periodo_cupon)
    periodos_comp=dias_val_venc//(360/periodo_cupon)
    
    V_int = 1 / (1 + ytm_periodo) ** (np.arange(periodos_comp + 1) + periodo_frac)
    V_risk = (1-h) ** np.arange(periodos_comp + 2)
    V_tot = V_risk[1:] * V_int
    V_dif = V_risk[:-1] * V_int
    
    Precio_Sucio = sum(cupon*V_tot)+(sum(V_dif)* h*R+V_tot[-1])*nocional
    
    return Precio_Sucio

Probamos nuestra funcion con el ejercicio en clase del bono de PEMEX

In [95]:
prueba_PEMEX=PrecioSucio(datetime(2020,4,8),datetime(2027,3,13),100,.0855079,.065)
prueba_PEMEX

89.88404657685007

Definimos la funcion proba_incumplimiento la cual regresa el precio sucio y la h que tenemos que asumir para mitigar el riesgo

In [202]:
def proba_incumplimiento(fecha_valuacion, fecha_vencimiento, nocional, ytm, tasa_cupon, periodo_cupon = 2, h=0,R = 0.25, InterBank = 0.0067):
    
    h0=0
    constraints = [
     {"type": "eq", "fun": lambda h: PrecioSucio(fecha_valuacion,fecha_vencimiento,nocional,ytm,tasa_cupon) - PrecioSucio(fecha_valuacion,fecha_vencimiento,nocional,InterBank,tasa_cupon,h=h,R=R)}]

    Precio_Sucio = PrecioSucio(fecha_valuacion,fecha_vencimiento,nocional,ytm,tasa_cupon)
    min_h = minimize(lambda h: (PrecioSucio(fecha_valuacion,fecha_vencimiento,nocional,ytm,tasa_cupon) - 
                           PrecioSucio(fecha_valuacion,fecha_vencimiento,nocional,InterBank,tasa_cupon,h=h,R=R)),
                           h0, constraints = constraints)
    return {"Precio_Sucio":Precio_Sucio, "h":min_h.x}

Probamos nuestra funcion con el ejercicio en clase del bono de PEMEX

In [213]:
prueba_PEMEX2 = proba_incumplimiento(datetime(2020,4,8),datetime(2027,3,13),100,.0855079,.065)
prueba_PEMEX2

{'Precio_Sucio': 89.88404657685007, 'h': array([0.05022308])}

Definimos la funcion CVA para que dado el precio y tasa en Moneda Extranjera(USD), nocional, plazo y puntos del fwd
,volumen, IIE,y la h definida por porba_incumplimiento, nos regrese el monto de CVA para nuestras
dos variantes EPE y PFE. Asi como el precio fwd, y los precios regargados (precio recargado EPE=precio recargado PFE)

In [362]:
def CVA(proba_incumplimiento, precio_ME, tasa_ME, nocional_fwd, plazo_fwd, pts_fwd, vol, TIIE, amplitud = 1, n_sim= 10000,plazo=91):
        
    precio_fwd= precio_ME+pts_fwd/10000
    tasa_ML=(precio_fwd/precio_ME*(1 + tasa_ME*plazo/360)- 1)*(360/plazo)
    sim= (np.ones(n_sim) * precio_ME)[:,np.newaxis]
    spread_tasas= tasa_ML-tasa_ME
    periodos= np.arange(start = 0, stop = plazo, step = amplitud)

    for periodo in periodos:
        sim= np.c_[sim,(
                     sim[:,-1] * 
                     np.exp((spread_tasas- 0.5*vol**2)*(amplitud/360) + 
                            (vol*(amplitud/360)**.5) * 
                            np.random.normal(size=n_sim))
                     )]

    MtM = nocional_fwd*(sim[:,:-1]*np.exp(spread_tasas*(plazo-periodos)/360)-precio_fwd)*np.exp(-tasa_ML*(plazo-periodos)/360)
    EPE = np.apply_along_axis(lambda x: x[x>=0].mean(), 0, MtM)
    PFE = np.apply_along_axis(lambda x: x[x>=0][x[x>=0]>=np.percentile(x[x>=0],.95)].mean(), 0, MtM)
    V_int = 1 / ((1 + TIIE * plazo_fwd/360)**((np.arange(len(EPE))+1)/plazo_fwd))
    V_risk = (1-(1-h)**((periodos+1)/180))-(1-(1-h)**(periodos/180))
    CVA_EPE = sum(EPE*V_int*V_risk)
    CVA_PFE = sum(PFE*V_int*V_risk)
    constraints = [    
    {"type": "eq", "fun": lambda precio_CVA: nocional_fwd*(precio_CVA-precio_fwd)*np.exp(-tasa_ML*plazo/360)-CVA_EPE}]
    
    opt  = minimize(lambda precio_CVA: nocional_fwd * (precio_CVA-precio_fwd)*np.exp(-tasa_ML*plazo/360)-CVA_EPE,
                               precio_fwd, constraints = constraints)

    precio_CVA_EPE = opt.x[0]
    
    constraints = [
    {"type": "eq", "fun": lambda precio_CVA: nocional_fwd*(precio_CVA-precio_fwd)*np.exp(-tasa_ML*plazo/360)-CVA_EPE}]

    opt  = minimize(lambda precio_CVA: nocional_fwd*(precio_CVA-precio_fwd)*np.exp(-tasa_ML * plazo/360)-CVA_PFE,
                               precio_fwd, constraints = constraints)

    precio_CVA_PFE = opt.x[0]
    
    return ({"CVA":{"EPE":f"${CVA_EPE}",
                    "PFE":f"${CVA_PFE}"},
                "Precios":
                        {"PrecioFwd":f"${precio_fwd}",
                         "PrecioRecargado_EPE":f"${precio_CVA_EPE}", 
                         "PrecioRecargado_PFE":f"${precio_CVA_PFE}"}})

In [365]:
h = prueba_PEMEX2["h"]

In [366]:
CVA_PEMEX=CVA(h,20.77,.007841,10000000,90,4118,.2,.0713,)
CVA_PEMEX

{'CVA': {'EPE': '$285818.68483738636', 'PFE': '$288573.3989102821'},
 'Precios': {'PrecioFwd': '$21.1818',
  'PrecioRecargado_EPE': '$21.211013196683517',
  'PrecioRecargado_PFE': '$21.21101319668028'}}